# DiSSECT Playground

In [ ]:
# Import dependencies
from dissect.analysis.data_processing import load_curves, load_trait, find_outliers
from dissect.analysis.visualization import violin

In [ ]:
# Define your data source or use the default
SOURCE = 'https://dissect.crocs.fi.muni.cz/static/data/'

In [ ]:
# Load curves
curves_source = load_curves(SOURCE)
curves_source

In [ ]:
# Filter curves
curves = curves_source.copy()
curves = curves[curves["bitlength"] == 224] # Filter curves with bitlength 224
curves = curves[curves["cofactor"] == 1] # Filter curves with cofactor 1
# curves = curves[curves["curve"].str.contains("x962")] # Filter curves simulated with x962 method
curves.reset_index(inplace=True, drop=True)
curves

In [ ]:
# Load trait results
trait_source = load_trait(SOURCE, "a05") # Select trait
trait_source

In [ ]:
# Filter trait results
trait = curves.merge(trait_source, "inner", on="curve", copy=True)
trait = trait[trait["l"] == 5] # Select params
trait.reset_index(inplace=True, drop=True)
trait

In [ ]:
violin(trait, "least")

In [ ]:
find_outliers(trait, ["least", "full"])